### Importing Libraries

In [ ]:
!pip install imbalanced-learn

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN
import pickle

# 1. CARGA DE DATOS
df = pd.read_csv("TelcoCustomer.csv")
telco_base_data = pd.read_csv('Telco-Customer-Churn.csv')
customer_ids = telco_base_data['customerID']

x = df.drop('Churn', axis=1)
y = df['Churn']

# 2. ENTRENAMIENTO DEL MODELO
print("✅ Entrenando modelo...")
sm = SMOTEENN(random_state=42)
X_resampled, y_resampled = sm.fit_resample(x, y)

xr_train, xr_test, yr_train, yr_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

model_rf = RandomForestClassifier(
    n_estimators=50,
    criterion='gini',
    random_state=100,
    max_depth=6,
    min_samples_leaf=8,
    n_jobs=-1
)
model_rf.fit(xr_train, yr_train)

# 3. PREDICCIONES
df_final = df.copy()
df_final['Churn_Probability'] = model_rf.predict_proba(x)[:, 1]

def definir_riesgo(prob):
    if prob < 0.40: return 'Bajo Riesgo'
    elif prob < 0.70: return 'Riesgo Medio'
    else: return 'Alto Riesgo'

df_final['Risk_Category'] = df_final['Churn_Probability'].apply(definir_riesgo)

# 4. FACTORES
print("🔍 Calculando factores MEJORADOS...")

def calcular_factores_mejorados(df):
    factor_1 = []
    factor_2 = []
    factor_3 = []

    for i in range(len(df)):
        row = df.iloc[i]
        factores = []

        # PERFIL 1: CLIENTE DE ALTO VALOR EN RIESGO
        if row['MonthlyCharges'] > 75 and row.get('Contract_Month-to-month', 0) == 1:
            if row.get('InternetService_Fiber optic', 0) == 1:
                factores.append('Cliente con Pago Elevado, Fibra Optica y Contrato Mensual')
            else:
                factores.append('Cliente con Pago Elevado y Contrato Mensual')

        # PERFIL 2: CLIENTE NUEVO PROBLEMÁTICO
        elif row['tenure_group_1 - 12'] == 1:
            if row['MonthlyCharges'] > 65:
                factores.append('Cliente Nuevo con Pago Elevado')
            elif row.get('TechSupport_No', 0) == 1:
                factores.append('Cliente Nuevo Sin Seguimiento')

        # PERFIL 3: ADULTO MAYOR VULNERABLE
        elif row['SeniorCitizen'] == 1:
            if row.get('Dependents_No', 0) == 1 and row.get('Partner_No', 0) == 1:
                factores.append('Cliente de Tercera Edad que vive solo y necesita Asistencia Adicional')
            else:
                factores.append('Cliente de Tercera Edad que podría necesitar Asistencia Adicional')

        # PERFIL 4: PROBLEMAS DE SERVICIO
        if row.get('OnlineSecurity_No', 0) == 1 and row.get('OnlineBackup_No', 0) == 1:
            factores.append('Cliente que No contrato Servicios Adicionales de Seguridad o Protección de Dispositivos')
        elif row.get('TechSupport_No', 0) == 1 and row['MonthlyCharges'] > 50:
            factores.append('Cliente que cumple con pagar, pero el proceso le resulta complicado')

        # PERFIL 5: MÉTODOS DE PAGO PROBLEMÁTICOS
        if row.get('PaymentMethod_Electronic check', 0) == 1:
            factores.append('Cliente que usa Método Electrónico para pagar')

        # FACTORES DE CONTEXTO (si no hay suficientes)
        if len(factores) < 3:
            if row.get('StreamingTV_Yes', 0) == 1 and row.get('StreamingMovies_Yes', 0) == 1:
                factores.append('Consumidor intensivo de video/música')
            elif row.get('PaperlessBilling_Yes', 0) == 1:
                factores.append('Cliente que prefiere Factura Digital')
            elif row['TotalCharges'] > 3000:
                factores.append('Cliente Antiguo')

        # GARANTIZAR 3 FACTORES ESPECÍFICOS
        while len(factores) < 3:
            if row['Churn_Probability'] > 0.6:
                factores.append('Alta Propensión a Cancelar')

        factor_1.append(factores[0] if len(factores) > 0 else 'Analizar Perfil')
        factor_2.append(factores[1] if len(factores) > 1 else 'Segmento Estándar')
        factor_3.append(factores[2] if len(factores) > 2 else 'Monitorizar')

    return factor_1, factor_2, factor_3

factor_1, factor_2, factor_3 = calcular_factores_mejorados(df_final)

# Crear DataFrame de factores
factors_df = pd.DataFrame({
    'Factor_1': factor_1,
    'Factor_2': factor_2,
    'Factor_3': factor_3
})

# 5. UNIR PRIMERO LOS FACTORES AL DATAFRAME PRINCIPAL
print("🔄 Uniendo factores al dataset...")
df_con_factores = pd.concat([df_final.reset_index(drop=True), factors_df.reset_index(drop=True)], axis=1)

# 6. RECOMENDACIONES MEJORADAS - MÁS ACCIONABLES
print("🎯 Generando recomendaciones MEJORADAS...")

def recomendacion_mejorada(row):
    prob = row['Churn_Probability']

    if prob < 0.40:
      return """Cliente Seguro
Sugerencia: Comunicación mensual + Ofertas cross-selling"""

    factores = [row['Factor_1'], row['Factor_2'], row['Factor_3']]

    # RECOMENDACIONES ESPECÍFICAS POR PERFIL
    if 'Cliente con Pago Elevado, Fibra Optica y Contrato Mensual' in factores:
        return "Oferta: Contrato 1 año + 2 meses gratis + Router premium"

    elif 'Cliente Nuevo con Pago Elevado' in factores:
        return "Programa: Mentor asignado + Revisión gratuita + Encuesta día 30"

    elif 'Cliente de Tercera Edad que vive solo y necesita Asistencia Adicional' in factores:
        return "Plan: Soporte prioritario 24/7 + Visita técnica gratis + Familiar referente"

    elif 'Cliente que No contrato Servicios Adicionales de Seguridad o Protección de Dispositivos' in factores:
        return "Promo: Pack Seguridad $5/mes x 6 meses + Instalación gratis"

    elif 'Cliente que usa Método Electrónico para pagar' in factores:
        return "Incentivo: $20 crédito por activar pago automático"

    elif 'Cliente que cumple con pagar, pero el proceso le resulta complicado' in factores:
        return "Incluir: Soporte técnico premium 3 meses gratis"

    # RECOMENDACIONES POR PROBABILIDAD
    elif prob > 0.70:
        return "ACCION INMEDIATA: Llamada gerencial + Oferta personalizada"
    elif prob > 0.50:
        return "Email personalizado + Oferta 15% descuento x 3 meses"
    else:
        return "Encuesta de satisfacción + Oferta servicio adicional"

# Aplicar recomendaciones MEJORADAS al dataframe que YA tiene los factores
df_con_factores['Marketing_Action'] = df_con_factores.apply(recomendacion_mejorada, axis=1)

# 7. AGREGAR CUSTOMER ID Y GUARDAR
df_export = df_con_factores.copy()
df_export.insert(0, 'customerID', customer_ids)

nombre_archivo = "Clientes_Analisis_Mejorado.csv"
df_export.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

# 8. RESULTADOS DETALLADOS
print(f"📊 Archivo: {nombre_archivo}")

print("\n🔍 MUESTRA DE RESULTADOS MEJORADOS:")
print("="*60)

# Mostrar ejemplos variados
muestra_clientes = df_export.sample(min(8, len(df_export)), random_state=42)
for i, (_, cliente) in enumerate(muestra_clientes.iterrows(), 1):
    print(f"👤 Cliente {i} | {cliente['Risk_Category']} | Probabilidad: {cliente['Churn_Probability']:.1%}")
    print(f"   📋 Factores:")
    print(f"     1. {cliente['Factor_1']}")
    print(f"     2. {cliente['Factor_2']}")
    print(f"     3. {cliente['Factor_3']}")
    print(f"   🎯 Acción: {cliente['Marketing_Action']}")
    print()

print("📈 ESTADÍSTICAS DETALLADAS:")
print("="*40)
print(f"📊 Total clientes analizados: {len(df_export):,}")
print(f"🔴 Alto Riesgo: {len(df_export[df_export['Risk_Category'] == 'Alto Riesgo']):,}")
print(f"🟡 Riesgo Medio: {len(df_export[df_export['Risk_Category'] == 'Riesgo Medio']):,}")
print(f"🟢 Bajo Riesgo: {len(df_export[df_export['Risk_Category'] == 'Bajo Riesgo']):,}")

print(f"\n🎯 VARIEDAD DE FACTORES:")
print("="*40)
print(f"• Factor 1: {df_export['Factor_1'].nunique()} tipos únicos")
print(f"• Factor 2: {df_export['Factor_2'].nunique()} tipos únicos")
print(f"• Factor 3: {df_export['Factor_3'].nunique()} tipos únicos")

print(f"\n📋 DISTRIBUCIÓN DE ACCIONES:")
print("="*40)
acciones_comunes = df_export['Marketing_Action'].value_counts().head(8)
for accion, count in acciones_comunes.items():
    print(f"• {accion}: {count} clientes")

print(f"\n🎭 PERFILES MÁS COMUNES:")
print("="*40)
factores_comunes = df_export['Factor_1'].value_counts().head(5)
for factor, count in factores_comunes.items():
    print(f"• {factor}: {count} clientes")

# Guardar modelo
filename = 'model_rf_churn_mejorado.sav'
pickle.dump(model_rf, open(filename, 'wb'))
print(f"\n🤖 Modelo guardado: {filename}")